# SWELL-KW GRU

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl
import pathlib

In [2]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
# Making sure edgeml is part of python path
sys.path.insert(0, '../../')
#For processing on CPU.
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

np.random.seed(42)
tf.set_random_seed(42)

# MI-RNN and EMI-RNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_GRU
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

Using TensorFlow backend.


In [4]:
# Network parameters for our LSTM + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 8
ORIGINAL_NUM_TIMESTEPS = 20
NUM_FEATS = 22
FORGET_BIAS = 1.0
NUM_OUTPUT = 2
USE_DROPOUT = True
KEEP_PROB = 0.75

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 2
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 30
LEARNING_RATE=0.001

# A staging direcory to store models
MODEL_PREFIX = '/home/sf/data/SWELL-KW/models/GRU/model-gru'

# Loading Data

In [5]:
# Loading the data
x_train, y_train = np.load('/home/sf/data/SWELL-KW/8_3/x_train.npy'), np.load('/home/sf/data/SWELL-KW/8_3/y_train.npy')
x_test, y_test = np.load('/home/sf/data/SWELL-KW/8_3/x_test.npy'), np.load('/home/sf/data/SWELL-KW/8_3/y_test.npy')
x_val, y_val = np.load('/home/sf/data/SWELL-KW/8_3/x_val.npy'), np.load('/home/sf/data/SWELL-KW/8_3/y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (3679, 5, 8, 22)
y_train shape is: (3679, 5, 2)
x_test shape is: (409, 5, 8, 22)
y_test shape is: (409, 5, 2)


# Computation Graph

In [6]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
    if inference is False:
        feedDict = {self._emiGraph.keep_prob: keep_prob}
    else:
        feedDict = {self._emiGraph.keep_prob: 1.0}
    return feedDict
    
EMI_GRU._createExtendedGraph = createExtendedGraph
EMI_GRU._restoreExtendedGraph = restoreExtendedGraph

if USE_DROPOUT is True:
    EMI_Driver.feedDictFunc = feedDictFunc

In [7]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiGRU = EMI_GRU(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy',
                         stepSize=LEARNING_RATE)

In [8]:
tf.reset_default_graph()
g1 = tf.Graph()    
with g1.as_default():
    # Obtain the iterators to each batch of the data
    x_batch, y_batch = inputPipeline()
    # Create the forward computation graph based on the iterators
    y_cap = emiGRU(x_batch)
    # Create loss graphs and training routines
    emiTrainer(y_cap, y_batch)

# EMI Driver

In [9]:
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiGRU, emiTrainer)

emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                      y_train=y_train, bag_train=BAG_TRAIN,
                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                      fracEMI=0.5, updatePolicy='top-k', k=1)

Update policy: top-k
Training with MI-RNN loss for 15 rounds
Round: 0
Epoch   1 Batch   110 (  225) Loss 0.08950 Acc 0.63125 | Val acc 0.58680 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1000
Epoch   1 Batch   110 (  225) Loss 0.08044 Acc 0.61875 | Val acc 0.60636 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1001
Epoch   1 Batch   110 (  225) Loss 0.08588 Acc 0.59375 | Val acc 0.61369 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1002
Epoch   1 Batch   110 (  225) Loss 0.06992 Acc 0.70625 | Val acc 0.63570 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1003
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1003
Round: 1
Epoch   1 Batch   110 (  225) Loss 0.07058 Acc 0.65625 | Val acc 0.64792 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1004
Epoch   1 Batch   110 (  225) Loss 0.06464 Acc 0.71250 | Val acc 0.667

Round: 23
Epoch   1 Batch   110 (  225) Loss 0.39712 Acc 0.86250 | Val acc 0.85330 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1092
Epoch   1 Batch   110 (  225) Loss 0.35570 Acc 0.89375 | Val acc 0.84841 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1093
Epoch   1 Batch   110 (  225) Loss 0.35381 Acc 0.88750 | Val acc 0.84841 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1094
Epoch   1 Batch   110 (  225) Loss 0.37666 Acc 0.85625 | Val acc 0.84352 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1095
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1092
Round: 24
Epoch   1 Batch   110 (  225) Loss 0.38746 Acc 0.88750 | Val acc 0.85575 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model-gru, global_step 1096
Epoch   1 Batch   110 (  225) Loss 0.36476 Acc 0.91250 | Val acc 0.84597 | Model saved to /home/sf/data/SWELL-KW/models/GRU/model

# Evaluating the  trained model

In [10]:
# Early Prediction Policy: We make an early prediction based on the predicted classes
#     probability. If the predicted class probability > minProb at some step, we make
#     a prediction at that step.
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
    predictionStep = predictionStep + 1
    predictionStep = np.reshape(predictionStep, -1)
    totalSteps = np.sum(predictionStep)
    maxSteps = len(predictionStep) * numTimeSteps
    savings = 1.0 - (totalSteps / maxSteps)
    if returnTotal:
        return savings, totalSteps
    return savings

In [11]:
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
total_savings = mi_savings + (1 - mi_savings) * emi_savings
print('Savings due to MI-RNN : %f' % mi_savings)
print('Savings due to Early prediction: %f' % emi_savings)
print('Total Savings: %f' % (total_savings))

Accuracy at k = 2: 0.842466
Savings due to MI-RNN : 0.600000
Savings due to Early prediction: 0.162573
Total Savings: 0.665029


In [12]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.831703   0.831557   0.835146   0.832988   0.831703   0.831703   
1    2  0.842466   0.842440   0.842466   0.842688   0.842466   0.842466   
2    3  0.846380   0.846156   0.846630   0.845956   0.846380   0.846380   
3    4  0.847358   0.846679   0.850044   0.846213   0.847358   0.847358   
4    5  0.839530   0.838276   0.845458   0.837832   0.839530   0.839530   

   micro-rec  fscore_01  
0   0.831703   0.836502  
1   0.842466   0.840436  
2   0.846380   0.840285  
3   0.847358   0.836478  
4   0.839530   0.824034  
Max accuracy 0.847358 at subsequencelength 4
Max micro-f 0.847358 at subsequencelength 4
Micro-precision 0.847358 at subsequencelength 4
Micro-recall 0.847358 at subsequencelength 4
Max macro-f 0.846679 at subsequencelength 4
macro-precision 0.850044 at subsequencelength 4
macro-recall 0.846213 at subsequencelength 4


## Picking the best model

In [18]:
devnull = open(os.devnull, 'r')
for val in modelStats:
    round_, acc, modelPrefix, globalStep = val
    emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)

    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
    print(', Test Accuracy (k = %d): %f, ' % (4,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
    mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
    emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
    total_savings = mi_savings + (1 - mi_savings) * emi_savings
    print("Total Savings: %f" % total_savings)

INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1003
Round:  0, Validation accuracy: 0.6357, Test Accuracy (k = 4): 0.667319, Total Savings: 0.602926
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1007
Round:  1, Validation accuracy: 0.6993, Test Accuracy (k = 4): 0.693738, Total Savings: 0.603063
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1010
Round:  2, Validation accuracy: 0.7042, Test Accuracy (k = 4): 0.685910, Total Savings: 0.603875
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1014
Round:  3, Validation accuracy: 0.7311, Test Accuracy (k = 4): 0.693738, Total Savings: 0.603806
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/GRU/model-gru-1019
Round:  4, Validation accuracy: 0.7506, Test Accuracy (k = 4): 0.709393, Total Savings: 0.604521
INFO:tensorflow:Restoring parameters from /home/sf/data/SWEL

In [14]:
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 64, #subinstance length.
    "ORIGINAL_NUM_TIMESTEPS" : 128,
    "NUM_FEATS" : 16,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 5,
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "LEARNING_RATE" : 0.001,
    "MODEL_PREFIX" : '/home/sf/data/DREAMER/Dominance/model-gru'
}

In [15]:
gru_dict = {**params}
gru_dict["k"] = k
gru_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
gru_dict["total_savings"] = total_savings
gru_dict["y_test"] = BAG_TEST
gru_dict["y_pred"] = bagPredictions

# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
print (tabulate(df, headers=list(df.columns), tablefmt='grid'))

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.831703   0.831557   0.835146   0.832988   0.831703   0.831703   
1    2  0.842466   0.842440   0.842466   0.842688   0.842466   0.842466   
2    3  0.846380   0.846156   0.846630   0.845956   0.846380   0.846380   
3    4  0.847358   0.846679   0.850044   0.846213   0.847358   0.847358   
4    5  0.839530   0.838276   0.845458   0.837832   0.839530   0.839530   

   micro-rec  fscore_01  
0   0.831703   0.836502  
1   0.842466   0.840436  
2   0.846380   0.840285  
3   0.847358   0.836478  
4   0.839530   0.824034  
Max accuracy 0.847358 at subsequencelength 4
Max micro-f 0.847358 at subsequencelength 4
Micro-precision 0.847358 at subsequencelength 4
Micro-recall 0.847358 at subsequencelength 4
Max macro-f 0.846679 at subsequencelength 4
macro-precision 0.850044 at subsequencelength 4
macro-recall 0.846213 at subsequencelength 4
+----+-------+----------+-------------+-------------+-------------+-------

In [16]:
dirname = "/home/sf/data/SWELL/GRU/"
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(gru_dict,open(dirname  + filename + ".pkl",mode='wb'))

Results for this run have been saved at /home/sf/data/SWELL/GRU/ .


In [17]:
dirname+filename+'.pkl'

'/home/sf/data/SWELL/GRU/2019-9-3|15-1.pkl'